In [1]:
import numpy as np

In [2]:
train_df = np.load('../data/preprocessed/train_data.npz')
test_df = np.load('../data/preprocessed/test_data.npz')

In [3]:
train_df['x_train'].shape
# test_df['x_test'].shape
# test_df['y_test'].shape

(89996, 50)

In [4]:
X_train = train_df['x_train']
y_train = train_df['y_train']
X_test = test_df['x_test']
y_test = test_df['y_test']


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_tr_np = X_train
y_tr_np = y_train
X_te_np = X_test
y_te_np = y_test

clf = LogisticRegression(max_iter=1000)
clf.fit(X_tr_np, y_tr_np)

y_pred = clf.predict(X_te_np)
print(classification_report(y_te_np, y_pred))


              precision    recall  f1-score   support

           0       0.78      0.99      0.87      1338
           1       0.84      1.00      0.91       847
           2       0.12      0.03      0.04       339
           3       0.02      0.01      0.01       634
           4       0.56      0.21      0.30      1035
           5       0.34      0.70      0.45       592
           6       0.00      0.00      0.00       741
           7       0.00      0.00      0.00       421
           8       0.46      0.87      0.60      1233

    accuracy                           0.54      7180
   macro avg       0.35      0.42      0.36      7180
weighted avg       0.44      0.54      0.46      7180



c:\Users\ADMIN\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ADMIN\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ADMIN\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.linear(x)
    
input_dim = X_train_tensor.shape[1] 
num_classes = len(torch.unique(y_train_tensor))
model = LogisticRegressionModel(input_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

epochs = 500
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    # print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    preds = outputs.argmax(dim=1)

y_true = y_test_tensor.cpu().numpy()
y_pred = preds.cpu().numpy()
print(classification_report(y_true, y_pred))
